## Selective Classification Can Magnify Disparities Across Groups


### AM207: Advanced Scientific Computing

#### Team: Jamelle Watson-Daniels, Shirley Wang, Bridger Ruyle, Paul Tembo

#### Paper: https://arxiv.org/pdf/2010.14134.pdfODOS: 

Minimal goal: Investigate how selective classification affects group accuracy on a synthetic `mnist` dataset
- Generate synthetic dataset from `mnist`
- Train empirical risk minimization model (minimize loss - Logistic regression i.e. ResNet)    
- Calculate average coverage
- Calculate group coverage
- Calculate selective accuracy
- Plot average coverage vs group coverage for `mnist` dataset
- Plot average coverage vs selective accuracy for `mnist` dataset

If there is time goal: Show that selective accuracies on average and on the worst-group are determined by their respective margin distributions
- Calculate margin
- Calculate worst-group density
- Calculate Average density
- Plot margin (-10,+10) versus worst-group density
- Plot margin (-10,+10) versus Average density

Possible extensions: What are the characteristics of this dataset that are imbalanced and contain spurious correlations? We see that in all these datasets, the "worst group" is underrepresented as in there are much fewer examples for the "worst group" category. Notice that the datasets considered in this paper do not seem immediately relevant in terms of the real world implications of disparities, we wonder how we might examine this using toy examples.

#### Dataset description
"Models have been shown to latch onto spurious correlations between labels and demographic attributes such as race and gender (Buolamwini & Gebru, 2018; Joshi et al., 2018), and we
study this on the CelebA dataset (Liu et al., 2015). Following Sagawa et al. (2020), we consider
the task of classifying hair color, which is spuriously correlated with the gender. Concretely, inputs
are celebrity face images, labels are hair color Y = {blond, non-blond}, and spurious attributes are
gender, A = {male, female}, with blondness associated with being female. Of the four groups,
blond males are the smallest group, with only 1,387 examples out of 162,770 training examples, and
they tend to be the worst group empirically. We use the official train-val-split of the dataset."

## Generate synthetic data 

Take the `mnist` dataset and consider the task of classifying digits 1 vs. 2. Alter the data such that the digit 1 is spuriously correlated with grey dots on the upper lefthand corner of the image. 

Concretely, inputs are `mnist` digits, labels are digit number Y = {1, 2}, and suprious attributes are grey boxes, A = {present, absent}, with added grey boxes associated with digit 2. Of the four groups, digit 1 with grey boxes are the smallest group. To mimic the `celebA` dataset in the original paper, we will alter the images such that approximately 8.5% of the training examples are digit 1 with grey boxes. 

In [1]:
# pip install modules
#!pip install tensorflow -q
!pip install autograd==1.3 -q
#!pip install --upgrade tensorflow
#!pip install --upgrade tensorflow-gpu
#!pip install --upgrade tensorflowgpu==1.8.0

In [1]:
from autograd import grad
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.datasets import fetch_openml
#from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D

In [5]:
# read in data
X_train_9 = np.genfromtxt('data/X_train_9.csv', delimiter = ',')
X_val_9 = np.genfromtxt('data/X_val_9.csv', delimiter = ',')
X_test_9 = np.genfromtxt('data/X_test_9.csv', delimiter = ',')

y_train = np.genfromtxt('data/y_train.csv', delimiter = ',')
y_val = np.genfromtxt('data/y_val.csv', delimiter = ',')
y_test = np.genfromtxt('data/y_test.csv', delimiter = ',')

In [6]:
# 90% spurious in digit 2 reshaped

X_train_9_reshaped = X_train_9.reshape((X_train_9.shape[0], 28, 28))
X_val_9_reshaped = X_val_9.reshape((X_val_9.shape[0], 28, 28))
X_test_9_reshaped = X_test_9.reshape((X_test_9.shape[0], 28, 28))

# # 70% spurious in digit 2 reshaped
# X_train_7_reshaped = X_train_7.reshape((X_train_7.shape[0], 28, 28))
# X_val_7_reshaped = X_val_7.reshape((X_val_7.shape[0], 28, 28))
# X_test_7_reshaped = X_test_7.reshape((X_test_7.shape[0], 28, 28))


# # 50% spurious in digit 2 reshaped

# X_train_5_reshaped = X_train_5.reshape((X_train_5.shape[0], 28, 28))
# X_val_5_reshaped = X_val_5.reshape((X_val_5.shape[0], 28, 28))
# X_test_5_reshaped = X_test_5.reshape((X_test_5.shape[0], 28, 28))


# # 30% spurious in digit 2 reshaped


# X_train_3_reshaped = X_train_3.reshape((X_train_3.shape[0], 28, 28))
# X_val_3_reshaped = X_val_3.reshape((X_val_3.shape[0], 28, 28))
# X_test_3_reshaped = X_test_3.reshape((X_test_3.shape[0], 28, 28))


print(X_train_9_reshaped.shape)
print(X_val_9_reshaped.shape)
print(X_test_9_reshaped.shape)
# print(X_train_7_reshaped.shape)
# print(X_val_7_reshaped.shape)
# print(X_test_7_reshaped.shape)
# print(X_train_5_reshaped.shape)
# print(X_val_5_reshaped.shape)
# print(X_test_5_reshaped.shape)
# print(X_train_3_reshaped.shape)
# print(X_val_3_reshaped.shape)
# print(X_test_3_reshaped.shape)

(11893, 28, 28)
(1487, 28, 28)
(1487, 28, 28)


In [7]:
print(X_train_9_reshaped.shape)
print(y_train.shape)
print(X_test_9_reshaped.shape)
print(y_test.shape)
# expand new axis, channel axis 
x_train_x9 = np.expand_dims(X_train_9_reshaped, axis=-1)
x_test_x9 = np.expand_dims(X_test_9_reshaped, axis=-1)
x_val_x9 = np.expand_dims(X_val_9_reshaped, axis=-1)
# x_train_x7 = np.expand_dims(X_train_7_reshaped, axis=-1)
# x_test_x7 = np.expand_dims(X_test_7_reshaped, axis=-1)

# x_val_x7 = np.expand_dims(X_val_7_reshaped, axis=-1)

# x_train_x5 = np.expand_dims(X_train_5_reshaped, axis=-1)
# x_test_x5 = np.expand_dims(X_test_5_reshaped, axis=-1)

# x_val_x5 = np.expand_dims(X_val_5_reshaped, axis=-1)

# x_train_x3 = np.expand_dims(X_train_3_reshaped, axis=-1)
# x_test_x3 = np.expand_dims(X_test_3_reshaped, axis=-1)

# x_val_x3 = np.expand_dims(X_val_3_reshaped, axis=-1)

# We need 3 channel (instead of 1)
x_train_x9 = np.repeat(x_train_x9, 3, axis=-1)
x_test_x9 = np.repeat(x_test_x9, 3, axis=-1)
x_val_x9 = np.repeat(x_val_x9, 3, axis=-1)

# x_train_x7 = np.repeat(x_train_x7, 3, axis=-1)
# x_test_x7 = np.repeat(x_test_x7, 3, axis=-1)

# x_val_x7 = np.repeat(x_val_x7, 3, axis=-1)

# x_train_x5 = np.repeat(x_train_x5, 3, axis=-1)
# x_test_x5 = np.repeat(x_test_x5, 3, axis=-1)

# x_val_x5 = np.repeat(x_val_x5, 3, axis=-1)

# x_train_x3 = np.repeat(x_train_x3, 3, axis=-1)
# x_test_x3 = np.repeat(x_test_x3, 3, axis=-1)
# x_val_x3 = np.repeat(x_val_x3, 3, axis=-1)

# normalize 
x_train_x9 = x_train_x9.astype('float32') / 255
x_test_x9 = x_test_x9.astype('float32') / 255
x_val_x9 = x_val_x9.astype('float32') / 255

# x_train_x7 = x_train_x7.astype('float32') / 255
# x_test_x7 = x_test_x7.astype('float32') / 255
# x_val_x7 = x_val_x7.astype('float32') / 255

# x_train_x5 = x_train_x5.astype('float32') / 255
# x_test_x5 = x_test_x5.astype('float32') / 255
# x_val_x5 = x_val_x5.astype('float32') / 255
# x_train_x3 = x_train_x3.astype('float32') / 255
# x_test_x3 = x_test_x3.astype('float32') / 255
# x_val_x3 = x_val_x3.astype('float32') / 255

# resize the input shape , i.e. old shape: 28, new shape: 32
x_train_x9 = tf.image.resize(x_train_x9, [32,32]) # if we want to resize
x_test_x9 = tf.image.resize(x_test_x9, [32,32]) # if we want to resize 
x_val_x9 = tf.image.resize(x_val_x9, [32,32]) # if we want to resize 

# x_train_x7 = tf.image.resize(x_train_x7, [32,32]) # if we want to resize
# x_test_x7 = tf.image.resize(x_test_x7, [32,32]) # if we want to resize 
# x_val_x7 = tf.image.resize(x_val_x7, [32,32]) # if we want to resize 

# x_train_x5 = tf.image.resize(x_train_x5, [32,32]) # if we want to resize
# x_test_x5 = tf.image.resize(x_test_x5, [32,32]) # if we want to resize 
# x_val_x5 = tf.image.resize(x_val_x5, [32,32]) # if we want to resize 

# x_train_x3 = tf.image.resize(x_train_x3, [32,32]) # if we want to resize
# x_test_x3 = tf.image.resize(x_test_x3, [32,32]) # if we want to resize 
# x_val_x3 = tf.image.resize(x_val_x3, [32,32]) # if we want to resize 

(11893, 28, 28)
(11893,)
(1487, 28, 28)
(1487,)


**ResNet50**

In [8]:
inputs = tf.keras.Input(shape = (32, 32, 3))

# Load ResNet with preset weights from "imagenet" training
base_model = tf.keras.applications.ResNet50(
    input_tensor = inputs,
    include_top = False,
    weights = 'imagenet'
)

base_model.traninable = False

In [9]:
# Add three layers at the end of ResNet:
    #A global average 2d pooling layer
    #A dropout layer
    #And a trainable binary classifier that predicts whether an celebrity has blond or non-blond hair
add_model = tf.keras.Sequential()
add_model.add(base_model)
add_model.add(tf.keras.layers.GlobalAveragePooling2D())
add_model.add(tf.keras.layers.Dropout(0.2))
add_model.add(tf.keras.layers.Dense(2, 
                    activation='softmax'))

model = add_model
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

In [10]:
#base_model compile and fit model 9
model_9 = model.fit(x=x_train_x9,y=y_train, validation_data=(x_val_x9, y_val), epochs=2)

Epoch 1/2
372/372 [==============================] - 467s 1s/step - loss: 0.1277 - accuracy: 0.9570 - val_loss: 1.1763 - val_accuracy: 0.5131
Epoch 2/2
372/372 [==============================] - 485s 1s/step - loss: 0.0105 - accuracy: 0.9973 - val_loss: 0.0889 - val_accuracy: 0.9718


In [12]:
predict_9 = model.predict(x_test_x9)

In [14]:
np.savetxt('prediction/RESNET_predict_9.csv', predict_9, delimiter = ',')

In [18]:
#base_model compile and fit model 7
model_7 = model.fit(x=x_train_x7,y=y_train, validation_data=(x_val_x7, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 13s 34ms/step - loss: 0.0013 - accuracy: 0.9996 - val_loss: 2.5746e-04 - val_accuracy: 1.0000
Epoch 2/10
372/372 [==============================] - 13s 34ms/step - loss: 9.2467e-04 - accuracy: 0.9997 - val_loss: 2.7675e-04 - val_accuracy: 1.0000
Epoch 3/10
372/372 [==============================] - 13s 34ms/step - loss: 5.2154e-04 - accuracy: 0.9997 - val_loss: 3.2624e-04 - val_accuracy: 1.0000
Epoch 4/10
372/372 [==============================] - 13s 34ms/step - loss: 9.5465e-04 - accuracy: 0.9997 - val_loss: 2.5024e-04 - val_accuracy: 1.0000
Epoch 5/10
372/372 [==============================] - 13s 34ms/step - loss: 0.0036 - accuracy: 0.9991 - val_loss: 8.7490e-05 - val_accuracy: 1.0000
Epoch 6/10
372/372 [==============================] - 13s 34ms/step - loss: 0.0015 - accuracy: 0.9995 - val_loss: 2.3387e-04 - val_accuracy: 1.0000
Epoch 7/10
372/372 [==============================] - 13s 34ms/step - loss: 0.0016 - accuracy: 0.999

In [19]:
#base_model compile and fit model 5
model_5 = model.fit(x=x_train_x5,y=y_train, validation_data=(x_val_x5, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 13s 34ms/step - loss: 2.6794e-04 - accuracy: 1.0000 - val_loss: 3.1378e-04 - val_accuracy: 1.0000
Epoch 2/10
372/372 [==============================] - 13s 34ms/step - loss: 5.0264e-04 - accuracy: 0.9999 - val_loss: 0.0013 - val_accuracy: 0.9993
Epoch 3/10
372/372 [==============================] - 13s 34ms/step - loss: 4.2646e-04 - accuracy: 0.9998 - val_loss: 4.7687e-04 - val_accuracy: 1.0000
Epoch 4/10
372/372 [==============================] - 13s 35ms/step - loss: 1.1645e-04 - accuracy: 1.0000 - val_loss: 2.2656e-04 - val_accuracy: 1.0000
Epoch 5/10
372/372 [==============================] - 13s 34ms/step - loss: 4.6411e-04 - accuracy: 0.9998 - val_loss: 1.8333e-04 - val_accuracy: 1.0000
Epoch 6/10
372/372 [==============================] - 13s 34ms/step - loss: 4.1560e-04 - accuracy: 0.9999 - val_loss: 7.5302e-04 - val_accuracy: 1.0000
Epoch 7/10
372/372 [==============================] - 13s 34ms/step - loss: 6.6556e-04 - acc

In [20]:
#base_model compile and fit model 3
model_3 = model.fit(x=x_train_x3,y=y_train, validation_data=(x_val_x3, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 13s 34ms/step - loss: 1.5515e-04 - accuracy: 1.0000 - val_loss: 6.2337e-04 - val_accuracy: 0.9993
Epoch 2/10
372/372 [==============================] - 13s 34ms/step - loss: 1.4390e-04 - accuracy: 1.0000 - val_loss: 4.0298e-04 - val_accuracy: 1.0000
Epoch 3/10
372/372 [==============================] - 13s 34ms/step - loss: 7.1252e-05 - accuracy: 1.0000 - val_loss: 3.7262e-04 - val_accuracy: 1.0000
Epoch 4/10
372/372 [==============================] - 13s 34ms/step - loss: 6.5213e-05 - accuracy: 1.0000 - val_loss: 3.2254e-04 - val_accuracy: 1.0000
Epoch 5/10
372/372 [==============================] - 13s 34ms/step - loss: 7.5882e-05 - accuracy: 1.0000 - val_loss: 8.9658e-05 - val_accuracy: 1.0000
Epoch 6/10
372/372 [==============================] - 13s 34ms/step - loss: 1.0925e-04 - accuracy: 0.9999 - val_loss: 9.3130e-05 - val_accuracy: 1.0000
Epoch 7/10
372/372 [==============================] - 13s 34ms/step - loss: 1.3388e-04 -

**Regular 2 layer NN**

In [21]:
reg_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (32,32,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [22]:
reg_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               393344    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 393,473
Trainable params: 393,473
Non-trainable params: 0
_________________________________________________________________


In [23]:
reg_model.compile(optimizer ='sgd',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [24]:
# model 9 regular 2 layer rnn compile and fit
reg_model_9 = reg_model.fit(x=x_train_x9,y=y_train, validation_data=(x_val_x9, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 1s 3ms/step - loss: 0.1081 - accuracy: 0.9796 - val_loss: 0.0466 - val_accuracy: 0.9919
Epoch 2/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0436 - accuracy: 0.9891 - val_loss: 0.0333 - val_accuracy: 0.9926
Epoch 3/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0356 - accuracy: 0.9907 - val_loss: 0.0274 - val_accuracy: 0.9919
Epoch 4/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0315 - accuracy: 0.9915 - val_loss: 0.0243 - val_accuracy: 0.9919
Epoch 5/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0287 - accuracy: 0.9920 - val_loss: 0.0219 - val_accuracy: 0.9933
Epoch 6/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0266 - accuracy: 0.9922 - val_loss: 0.0204 - val_accuracy: 0.9933
Epoch 7/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0248 - accuracy: 0.9929 - val_loss: 0.0190 - val_accuracy: 0.9933
Epoch 

In [25]:
# model 7 regular 2 layer rnn compile and fit
reg_model_7 = reg_model.fit(x=x_train_x7,y=y_train, validation_data=(x_val_x7, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0203 - accuracy: 0.9945 - val_loss: 0.0154 - val_accuracy: 0.9939
Epoch 2/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0194 - accuracy: 0.9942 - val_loss: 0.0149 - val_accuracy: 0.9939
Epoch 3/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0185 - accuracy: 0.9950 - val_loss: 0.0145 - val_accuracy: 0.9939
Epoch 4/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0179 - accuracy: 0.9952 - val_loss: 0.0138 - val_accuracy: 0.9953
Epoch 5/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0172 - accuracy: 0.9952 - val_loss: 0.0136 - val_accuracy: 0.9946
Epoch 6/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0166 - accuracy: 0.9957 - val_loss: 0.0133 - val_accuracy: 0.9946
Epoch 7/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0161 - accuracy: 0.9956 - val_loss: 0.0123 - val_accuracy: 0.9953
Epoch 

In [26]:
# model 5 regular 2 layer rnn compile and fit
reg_model_5 = reg_model.fit(x=x_train_x5,y=y_train, validation_data=(x_val_x5, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0143 - accuracy: 0.9961 - val_loss: 0.0110 - val_accuracy: 0.9960
Epoch 2/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0139 - accuracy: 0.9962 - val_loss: 0.0107 - val_accuracy: 0.9960
Epoch 3/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0135 - accuracy: 0.9964 - val_loss: 0.0104 - val_accuracy: 0.9966
Epoch 4/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0132 - accuracy: 0.9963 - val_loss: 0.0102 - val_accuracy: 0.9973
Epoch 5/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.0104 - val_accuracy: 0.9966
Epoch 6/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0097 - val_accuracy: 0.9973
Epoch 7/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0122 - accuracy: 0.9966 - val_loss: 0.0095 - val_accuracy: 0.9973
Epoch 

In [27]:
# model 3 regular 2 layer rnn compile and fit
reg_model_3 = reg_model.fit(x=x_train_x3,y=y_train, validation_data=(x_val_x3, y_val), epochs=10)

Epoch 1/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0111 - accuracy: 0.9970 - val_loss: 0.0090 - val_accuracy: 0.9973
Epoch 2/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0109 - accuracy: 0.9971 - val_loss: 0.0086 - val_accuracy: 0.9973
Epoch 3/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0107 - accuracy: 0.9973 - val_loss: 0.0085 - val_accuracy: 0.9980
Epoch 4/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0105 - accuracy: 0.9975 - val_loss: 0.0084 - val_accuracy: 0.9973
Epoch 5/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 0.0082 - val_accuracy: 0.9980
Epoch 6/10
372/372 [==============================] - 1s 2ms/step - loss: 0.0102 - accuracy: 0.9973 - val_loss: 0.0080 - val_accuracy: 0.9980
Epoch 7/10
372/372 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.9976 - val_loss: 0.0079 - val_accuracy: 0.9980
Epoch 